## 5 Layer Neural Network for Binary Classification

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np


### Generate synthetic data

In [4]:
def generate_dataset():
    X, y = make_classification(
        n_samples=2000, n_features=10, n_informative=5, n_redundant=2,
        n_classes=2, random_state=42
    )
    return X, y

### Define the neural network class


In [5]:
class FiveLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FiveLayerNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()  # Output layer for binary classification
        )

    def forward(self, x):
        return self.network(x)


### Prepare data

In [6]:
X, y = generate_dataset()
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Model parameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1


### Initialize model, loss function, and optimizer 

In [7]:
model = FiveLayerNN(input_size, hidden_size, output_size)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Train the model 

In [8]:
# Training loop
def train_model(model, X_train, y_train, epochs=100):
    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Train the model
train_model(model, X_train, y_train, epochs=100)

Epoch [10/100], Loss: 0.6669
Epoch [20/100], Loss: 0.5884
Epoch [30/100], Loss: 0.4354
Epoch [40/100], Loss: 0.3084
Epoch [50/100], Loss: 0.2454
Epoch [60/100], Loss: 0.2103
Epoch [70/100], Loss: 0.1909
Epoch [80/100], Loss: 0.1742
Epoch [90/100], Loss: 0.1586
Epoch [100/100], Loss: 0.1407


### Test the model

In [9]:
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_labels = (y_pred >= 0.5).float()
    accuracy = accuracy_score(y_test.numpy(), y_pred_labels.numpy())
    print(f"Test Accuracy: {accuracy:.4f}")



Test Accuracy: 0.9100


### Save the test data and model 

In [10]:
# Define the file path
file_path = 'data/classification_1.txt'

# Combine features and output for each test instance
test_data = np.hstack((X_test, y_test.numpy().reshape(-1, 1)))

# Save to file without brackets
with open(file_path, 'w') as f:
    for row in test_data:
        # Write row as a comma-separated line without brackets
        f.write(", ".join(map(str, row)) + "\n")

print(f"Test data saved to {file_path}")

# Define the full path for saving the model
model_path = 'model/pth/classification_1_model.pth'

# Save the model parameters
torch.save(model.state_dict(), model_path)

print(f"Model saved to {model_path}")

# Define the path for the ONNX file
onnx_path = 'model/onnx/classification_1_model.onnx'

# Specify a dummy input to trace the model's structure
dummy_input = torch.randn(1, input_size)  # Adjust the shape based on your model's input requirements

# Export the model to ONNX format
torch.onnx.export(
    model,                        # Model to be exported
    dummy_input,                  # Dummy input tensor to define the input size
    onnx_path,                    # Path to save the ONNX model
    export_params=True,           # Store the trained parameter weights inside the model file
    opset_version=11,             # ONNX version to export to
    input_names=['input'],        # Name of the input layer
    output_names=['output'],      # Name of the output layer
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # Dynamic batching
)

print(f"Model saved in ONNX format to {onnx_path}")


Test data saved to data/classification_1.txt
Model saved to model/pth/classification_1_model.pth
Model saved in ONNX format to model/onnx/classification_1_model.onnx
